In [1]:
import pandas as pd
import numpy as np

In [9]:
image_folder_path = "C:/Users/louis/github_vs/113_data/2017"
df = pd.read_csv(image_folder_path+"/201707"+"/message_info.tsv",sep="\t")
df

,event_id,message_id,user_handle,sec_since_start
0,cl070909,0,system_1,0.0
1,cl070909,1,employee03119,9.0
2,cl070909,2,client,44.0
3,cl070909,3,employee03119,64.0
4,cl070909,4,system_8,70.0
...,...,...,...,...
116352,cl072426,91,client,2989.0
116353,cl072426,92,employee00064,3027.0
116354,cl072426,93,client,3057.0
116355,cl072426,94,employee00064,3112.0
